In [1]:
%load_ext autoreload
%autoreload 2

import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, os.path.join(parent_dir)) 

%matplotlib inline

In [2]:
import networkx as nx
import numpy as np
from edge_prop.data_loader import DataLoader
from edge_prop.models.dense_edge_propagation import DenseEdgeProp

In [6]:
g = nx.barabasi_albert_graph(100, 30)

NUM_CLASSES = 2
def int2onehot(i, max_int=NUM_CLASSES):
    x = [0] * max_int
    x[i] = 1
    return x

edge2label = {edge:int2onehot(np.random.randint(NUM_CLASSES)) for edge in g.edges()}
nx.set_edge_attributes(g,edge2label,'label')

In [7]:
A=nx.adj_matrix(g).toarray()

In [8]:
Y = np.zeros(shape=(A.shape[0], A.shape[1], NUM_CLASSES))
for (u,v), label in edge2label.items():
    Y[u,v] = label
    Y[v,u] = label

In [203]:
def our_method(A, Y):
    last_Y = Y.copy()
    for i in range(100):
        B = np.sum(np.dot(A, last_Y),axis=0)
        D = np.sum(A,axis=0)

        mat = (B[:,np.newaxis,:]+B[np.newaxis,:,:]) / (D[:,np.newaxis] + D[np.newaxis,:])[:,:,np.newaxis]

        mat[A==0] = 0
        mat[A!=0] = mat[A!=0]/np.sum(mat[A!=0],axis=-1)[:,np.newaxis]
        mat[mat!=mat] = 0

        last_Y = mat
    return last_Y

In [208]:
def their_method(A, Y):
    last_Y = np.sum(np.dot(A, Y),axis=0)
    last_Y = last_Y/np.sum(last_Y,axis=-1)[:,np.newaxis]
    last_Y[last_Y!=last_Y] = 0
    
    for i in range(100):
        B = np.dot(A, last_Y)
        D = np.sum(A,axis=0)
        last_Y = B/D[:,np.newaxis]
    
#     last_Y=last_Y[:,np.newaxis,:]*last_Y[np.newaxis,:,:]
    last_Y=last_Y[:,np.newaxis,:]+last_Y[np.newaxis,:,:]
    last_Y[A==0] = 0
    last_Y[A!=0] = last_Y[A!=0]/np.sum(last_Y[A!=0],axis=-1)[:,np.newaxis]
    last_Y[last_Y!=last_Y] = 0
    
    return last_Y

In [205]:
np.allclose(their_method(A, Y),our_method(A, Y))

False

In [209]:
their_method(A, Y)[:,:,0]

array([[0.        , 0.        , 0.        , ..., 0.50206147, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.50206147, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.50206147, 0.50206147, 0.        , ..., 0.        , 0.        ,
        0.50206147],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.50206147, 0.        ,
        0.        ]])

In [210]:
our_method(A, Y)[:,:,0]

array([[0.        , 0.        , 0.        , ..., 0.49784624, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.49784624, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.49784624, 0.49784624, 0.        , ..., 0.        , 0.        ,
        0.49784624],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.49784624, 0.        ,
        0.        ]])

### Experiment on real world data

In [48]:
path = r'C:\Users\Brus\Documents\repos\TemporalEdgeProp\data\soc-sign-Slashdot090221.txt'
graph, y_true = DataLoader(path).load_data(1000)

In [49]:
edge_prop = DenseEdgeProp(graph.y_attr, max_iter=5)
edge_prop.fit(graph)

C:\Users\Brus\Documents\repos\TemporalEdgeProp\edge_prop\models\dense_edge_propagation.py:119: RuntimeWarning: divide by zero encountered in true_divide
  mat = (B[:, np.newaxis, :] + B[np.newaxis, :, :]) / (D[:, np.newaxis] + D[np.newaxis, :])[:, :, np.newaxis]
C:\Users\Brus\Documents\repos\TemporalEdgeProp\edge_prop\models\dense_edge_propagation.py:119: RuntimeWarning: invalid value encountered in true_divide
  mat = (B[:, np.newaxis, :] + B[np.newaxis, :, :]) / (D[:, np.newaxis] + D[np.newaxis, :])[:, :, np.newaxis]
C:\Users\Brus\Documents\repos\TemporalEdgeProp\edge_prop\models\dense_edge_propagation.py:122: RuntimeWarning: invalid value encountered in true_divide
  mat[adj_mat != 0] = mat[adj_mat != 0] / np.sum(mat[adj_mat != 0], axis=-1, keepdims=True)


C:\Users\Brus\Documents\repos\TemporalEdgeProp\edge_prop\models\dense_edge_propagation.py:127: ConvergenceWarning: max_iter was reached without convergence
  warnings.warn("max_iter was reached without convergence", category=ConvergenceWarning)


In [54]:
y_pred = edge_prop.predict()
accuracy = (y_pred == y_true).sum() / len(y_true)
print(f"Accuracy: {accuracy}")

Accuracy: 0.15284931051669712
